In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [20]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [21]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 421us/step - 

Epoch 65/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.8381 - val_loss: 25.7362
Epoch 66/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.6463 - val_loss: 25.4927
Epoch 67/1000
1052/1052 [==============================] - 0s 96us/step - loss: 21.3050 - val_loss: 25.6081
Epoch 68/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.6601 - val_loss: 25.8391
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.8145 - val_loss: 27.3498
Epoch 70/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.3956 - val_loss: 25.2750
Epoch 71/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.0531 - val_loss: 26.9048
Epoch 72/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.4779 - val_loss: 25.8963
Epoch 73/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.8152 - val_loss: 25.8249
Epoch 74/1000
1052/1052 [===

1052/1052 [==============================] - 0s 90us/step - loss: 21.6037 - val_loss: 24.8948
Epoch 141/1000
1052/1052 [==============================] - 0s 94us/step - loss: 21.9101 - val_loss: 25.5084
Epoch 142/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.2467 - val_loss: 25.7282
Epoch 143/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.7225 - val_loss: 25.6158
Epoch 144/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.9507 - val_loss: 25.9725
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1929 - val_loss: 27.9161
Epoch 146/1000
1052/1052 [==============================] - 0s 83us/step - loss: 22.5821 - val_loss: 26.3166
Epoch 147/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.5141 - val_loss: 25.1135
Epoch 148/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.3857 - val_loss: 25.6928
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 88us/step - loss: 15.2364 - val_loss: 18.8330
Epoch 216/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.6019 - val_loss: 16.8284
Epoch 217/1000
1052/1052 [==============================] - 0s 87us/step - loss: 15.2204 - val_loss: 18.4591
Epoch 218/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.8378 - val_loss: 17.3913
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.5308 - val_loss: 16.7107
Epoch 220/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.7610 - val_loss: 20.7837
Epoch 221/1000
1052/1052 [==============================] - 0s 88us/step - loss: 15.3003 - val_loss: 18.1783
Epoch 222/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.3044 - val_loss: 16.9156
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.1028 - val_loss: 17.6531
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 88us/step - loss: 11.5808 - val_loss: 14.8764
Epoch 291/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.8661 - val_loss: 14.4103
Epoch 292/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.0722 - val_loss: 15.0484
Epoch 293/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.6769 - val_loss: 13.8859
Epoch 294/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.8099 - val_loss: 14.5923
Epoch 295/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.3375 - val_loss: 13.5286
Epoch 296/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.5563 - val_loss: 13.6425
Epoch 297/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.9381 - val_loss: 13.0867
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.6075 - val_loss: 17.1491
Epoch 299/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 10.0457 - val_loss: 11.3840
Epoch 746/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3528 - val_loss: 10.4604
Epoch 747/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3076 - val_loss: 10.1634
Epoch 748/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.6201 - val_loss: 11.6934
Epoch 749/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4293 - val_loss: 9.4933
Epoch 750/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8703 - val_loss: 10.5802
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3857 - val_loss: 9.5456
Epoch 752/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3051 - val_loss: 9.8660
Epoch 753/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6006 - val_loss: 9.6411
Epoch 754/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 87us/step - loss: 8.3406 - val_loss: 9.8632
Epoch 822/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.4553 - val_loss: 10.2981
Epoch 823/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2883 - val_loss: 9.8052
Epoch 824/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0816 - val_loss: 9.5507
Epoch 825/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1000 - val_loss: 9.4579
Epoch 826/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1758 - val_loss: 10.9020
Epoch 827/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1073 - val_loss: 10.4865
Epoch 828/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1880 - val_loss: 9.9406
Epoch 829/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1571 - val_loss: 9.3367
Epoch 830/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 90us/step - loss: 8.3595 - val_loss: 9.1668
Epoch 898/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2527 - val_loss: 10.4008
Epoch 899/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4444 - val_loss: 9.8244
Epoch 900/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1804 - val_loss: 9.4028
Epoch 901/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0817 - val_loss: 9.0651
Epoch 902/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6487 - val_loss: 9.3116
Epoch 903/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2643 - val_loss: 9.4744
Epoch 904/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1908 - val_loss: 9.2459
Epoch 905/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0261 - val_loss: 9.8684
Epoch 906/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 89us/step - loss: 8.0975 - val_loss: 9.1086
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4042 - val_loss: 9.2171
Epoch 975/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4330 - val_loss: 9.6773
Epoch 976/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9869 - val_loss: 10.1182
Epoch 977/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2811 - val_loss: 9.3736
Epoch 978/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2371 - val_loss: 10.7302
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5756 - val_loss: 9.7154
Epoch 980/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2091 - val_loss: 10.1273
Epoch 981/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4066 - val_loss: 9.5433
Epoch 982/1000
1052/1052 [=======================

[array([[ 1.92684   , -3.2819602 ,  0.16170907,  0.7451401 ,  3.684478  ],
        [-0.5120534 ,  0.08816968,  0.18943238,  1.4303167 , -0.20188612],
        [-0.39603817,  0.03589958,  0.2904678 ,  1.6199942 ,  0.6786771 ],
        [-0.02675521,  0.06636933, -0.12156886, -0.25519815, -0.1550963 ],
        [ 0.3170898 , -2.430267  ,  0.17560655, -0.32337746,  0.32187825]],
       dtype=float32),
 array([ 1.3062803, -4.8312626, -1.4084458,  0.7101072,  4.6671124],
       dtype=float32),
 array([[-0.26930195,  0.83421636,  0.66880596,  0.2204506 , -0.08666676,
          0.9608654 ,  0.7645459 ,  0.8993816 ,  0.32728025,  0.8621221 ],
        [-1.8216622 ,  1.875882  ,  1.1469307 ,  1.066795  , -2.2097967 ,
          1.9365895 ,  1.6413672 ,  1.1358248 ,  2.0512748 ,  1.6679956 ],
        [-1.9696206 ,  1.7040391 ,  1.6395023 ,  1.9156444 , -1.9847156 ,
          1.6884484 ,  1.217574  ,  1.0689583 ,  1.7076293 ,  1.5407608 ],
        [ 0.11542796,  0.43356326,  0.6252136 ,  0.4653967 ,  

In [22]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_adam_3rd.h5')

In [23]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, sgd, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 170us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 75us/step - loss: 19.5915 - val_loss: 24.9366
Epoch 68/1000
1052/1052 [==============================] - 0s 73us/step - loss: 19.6102 - val_loss: 24.9570
Epoch 69/1000
1052/1052 [==============================] - 0s 71us/step - loss: 19.5702 - val_loss: 24.9421
Epoch 70/1000
1052/1052 [==============================] - 0s 74us/step - loss: 19.5492 - val_loss: 24.9398
Epoch 71/1000
1052/1052 [==============================] - 0s 73us/step - loss: 19.5554 - val_loss: 24.9449
Epoch 72/1000
1052/1052 [==============================] - 0s 70us/step - loss: 19.5084 - val_loss: 24.9121
Epoch 73/1000
1052/1052 [==============================] - 0s 72us/step - loss: 19.4932 - val_loss: 24.8918
Epoch 74/1000
1052/1052 [==============================] - 0s 73us/step - loss: 19.4898 - val_loss: 24.9510
Epoch 75/1000
1052/1052 [==============================] - 0s 71us/step - loss: 19.4643 - val_loss: 24.8655
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 74us/step - loss: 18.8367 - val_loss: 24.4887
Epoch 143/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.8445 - val_loss: 24.4102
Epoch 144/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.8124 - val_loss: 24.4319
Epoch 145/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.8241 - val_loss: 24.4240
Epoch 146/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.8166 - val_loss: 24.3660
Epoch 147/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.8027 - val_loss: 24.3788
Epoch 148/1000
1052/1052 [==============================] - 0s 66us/step - loss: 18.8045 - val_loss: 24.3682
Epoch 149/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.8113 - val_loss: 24.4120
Epoch 150/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.7693 - val_loss: 24.3425
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 75us/step - loss: 18.2601 - val_loss: 23.4706
Epoch 218/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.2691 - val_loss: 23.4824
Epoch 219/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.2561 - val_loss: 23.4661
Epoch 220/1000
1052/1052 [==============================] - 0s 78us/step - loss: 18.2429 - val_loss: 23.4415
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.2315 - val_loss: 23.4007
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.2267 - val_loss: 23.4034
Epoch 223/1000
1052/1052 [==============================] - 0s 78us/step - loss: 18.2181 - val_loss: 23.3982
Epoch 224/1000
1052/1052 [==============================] - 0s 79us/step - loss: 18.2068 - val_loss: 23.3794
Epoch 225/1000
1052/1052 [==============================] - 0s 82us/step - loss: 18.1926 - val_loss: 23.3361
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 71us/step - loss: 17.4760 - val_loss: 21.9124
Epoch 293/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.4646 - val_loss: 21.8323
Epoch 294/1000
1052/1052 [==============================] - 0s 66us/step - loss: 17.4565 - val_loss: 21.8410
Epoch 295/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.4231 - val_loss: 21.7819
Epoch 296/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.4457 - val_loss: 21.7668
Epoch 297/1000
1052/1052 [==============================] - ETA: 0s - loss: 16.92 - 0s 71us/step - loss: 17.4331 - val_loss: 21.7576
Epoch 298/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.3985 - val_loss: 21.7669
Epoch 299/1000
1052/1052 [==============================] - 0s 71us/step - loss: 17.3957 - val_loss: 21.7262
Epoch 300/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.3824 - val_loss: 21.6869
Epoch 301/

...

1052/1052 [==============================] - 0s 74us/step - loss: 15.3183 - val_loss: 18.0799
Epoch 743/1000
1052/1052 [==============================] - 0s 73us/step - loss: 15.3183 - val_loss: 18.1147
Epoch 744/1000
1052/1052 [==============================] - 0s 75us/step - loss: 15.3287 - val_loss: 18.0951
Epoch 745/1000
1052/1052 [==============================] - 0s 75us/step - loss: 15.3161 - val_loss: 18.0628
Epoch 746/1000
1052/1052 [==============================] - 0s 74us/step - loss: 15.3166 - val_loss: 18.0980
Epoch 747/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.3052 - val_loss: 18.0676
Epoch 748/1000
1052/1052 [==============================] - 0s 71us/step - loss: 15.3099 - val_loss: 18.0666
Epoch 749/1000
1052/1052 [==============================] - 0s 74us/step - loss: 15.3084 - val_loss: 18.0789
Epoch 750/1000
1052/1052 [==============================] - 0s 73us/step - loss: 15.3269 - val_loss: 18.0172
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 68us/step - loss: 15.1406 - val_loss: 17.8673
Epoch 818/1000
1052/1052 [==============================] - 0s 71us/step - loss: 15.1452 - val_loss: 17.8779
Epoch 819/1000
1052/1052 [==============================] - 0s 68us/step - loss: 15.1397 - val_loss: 17.8702
Epoch 820/1000
1052/1052 [==============================] - 0s 72us/step - loss: 15.1438 - val_loss: 17.8379
Epoch 821/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.1344 - val_loss: 17.8667
Epoch 822/1000
1052/1052 [==============================] - 0s 66us/step - loss: 15.1353 - val_loss: 17.8609
Epoch 823/1000
1052/1052 [==============================] - 0s 72us/step - loss: 15.1406 - val_loss: 17.8589
Epoch 824/1000
1052/1052 [==============================] - 0s 73us/step - loss: 15.1280 - val_loss: 17.8372
Epoch 825/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.1379 - val_loss: 17.8659
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 69us/step - loss: 14.9917 - val_loss: 17.6613
Epoch 893/1000
1052/1052 [==============================] - 0s 72us/step - loss: 14.9942 - val_loss: 17.6914
Epoch 894/1000
1052/1052 [==============================] - 0s 67us/step - loss: 14.9919 - val_loss: 17.6961
Epoch 895/1000
1052/1052 [==============================] - 0s 72us/step - loss: 14.9800 - val_loss: 17.6623
Epoch 896/1000
1052/1052 [==============================] - 0s 73us/step - loss: 14.9769 - val_loss: 17.6747
Epoch 897/1000
1052/1052 [==============================] - 0s 74us/step - loss: 14.9993 - val_loss: 17.6554
Epoch 898/1000
1052/1052 [==============================] - 0s 73us/step - loss: 14.9739 - val_loss: 17.6546
Epoch 899/1000
1052/1052 [==============================] - 0s 66us/step - loss: 14.9738 - val_loss: 17.6704
Epoch 900/1000
1052/1052 [==============================] - 0s 72us/step - loss: 14.9683 - val_loss: 17.6519
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 69us/step - loss: 14.8436 - val_loss: 17.5201
Epoch 968/1000
1052/1052 [==============================] - 0s 72us/step - loss: 14.8404 - val_loss: 17.4933
Epoch 969/1000
1052/1052 [==============================] - 0s 69us/step - loss: 14.8428 - val_loss: 17.5505
Epoch 970/1000
1052/1052 [==============================] - 0s 70us/step - loss: 14.8407 - val_loss: 17.4983
Epoch 971/1000
1052/1052 [==============================] - 0s 69us/step - loss: 14.8403 - val_loss: 17.5064
Epoch 972/1000
1052/1052 [==============================] - 0s 71us/step - loss: 14.8399 - val_loss: 17.5130
Epoch 973/1000
1052/1052 [==============================] - 0s 70us/step - loss: 14.8335 - val_loss: 17.5283
Epoch 974/1000
1052/1052 [==============================] - 0s 68us/step - loss: 14.8271 - val_loss: 17.4926
Epoch 975/1000
1052/1052 [==============================] - 0s 72us/step - loss: 14.8357 - val_loss: 17.4972
Epoch 976/1000
1052/1052 [========

[array([[ 1.6106284 ,  0.06515915,  0.14764558, -0.02365688,  0.8669707 ],
        [ 1.0868931 ,  0.13901785, -0.00301819, -0.02235291,  0.15329546],
        [ 0.44952208,  0.15943629,  0.2522569 ,  0.12513375,  0.2537208 ],
        [-0.10319894,  0.18265331, -0.13179001,  0.01784311, -0.03227387],
        [-0.1455861 ,  0.01175106,  0.2629915 , -0.01273644,  0.47124058]],
       dtype=float32),
 array([ 0.19633093, -1.9195906 , -0.6959626 , -2.2038178 ,  1.2749046 ],
       dtype=float32),
 array([[-0.41821557,  0.01642537,  0.41227722,  0.9299322 , -0.30698398,
         -0.646844  ,  0.2780088 , -0.34349254, -0.14153719,  0.1007202 ],
        [-0.35933125,  1.1492908 ,  1.3593154 ,  1.5676874 , -0.34274447,
         -1.0212884 ,  0.8116155 , -1.0162488 , -0.07069721,  0.07800708],
        [-0.71646273,  0.67466563,  0.86960447,  1.4206023 ,  0.14678685,
         -1.1435691 ,  0.69368273, -0.7574048 , -0.22103702,  0.51806355],
        [ 0.08872592,  1.6197145 ,  1.6694177 ,  2.237666

In [24]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sgd_3rd.h5')

In [25]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, RMSprop, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 179us/step - l

1052/1052 [==============================] - 0s 82us/step - loss: 22.0712 - val_loss: 29.3447
Epoch 68/1000
1052/1052 [==============================] - 0s 79us/step - loss: 21.7341 - val_loss: 25.3078
Epoch 69/1000
1052/1052 [==============================] - 0s 82us/step - loss: 21.2344 - val_loss: 25.6408
Epoch 70/1000
1052/1052 [==============================] - 0s 82us/step - loss: 21.6286 - val_loss: 24.3973
Epoch 71/1000
1052/1052 [==============================] - 0s 78us/step - loss: 21.0779 - val_loss: 24.8684
Epoch 72/1000
1052/1052 [==============================] - 0s 80us/step - loss: 21.1574 - val_loss: 25.1976
Epoch 73/1000
1052/1052 [==============================] - 0s 76us/step - loss: 20.5294 - val_loss: 28.6937
Epoch 74/1000
1052/1052 [==============================] - 0s 75us/step - loss: 20.2125 - val_loss: 23.6324
Epoch 75/1000
1052/1052 [==============================] - 0s 78us/step - loss: 20.3960 - val_loss: 23.7130
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 81us/step - loss: 14.7531 - val_loss: 19.3287
Epoch 143/1000
1052/1052 [==============================] - 0s 80us/step - loss: 15.1169 - val_loss: 16.2198
Epoch 144/1000
1052/1052 [==============================] - 0s 77us/step - loss: 15.2059 - val_loss: 16.1376
Epoch 145/1000
1052/1052 [==============================] - 0s 79us/step - loss: 15.0214 - val_loss: 16.5450
Epoch 146/1000
1052/1052 [==============================] - 0s 78us/step - loss: 15.0577 - val_loss: 19.2735
Epoch 147/1000
1052/1052 [==============================] - 0s 80us/step - loss: 14.4671 - val_loss: 17.5173
Epoch 148/1000
1052/1052 [==============================] - 0s 81us/step - loss: 14.8615 - val_loss: 16.3924
Epoch 149/1000
1052/1052 [==============================] - 0s 74us/step - loss: 14.3289 - val_loss: 17.9686
Epoch 150/1000
1052/1052 [==============================] - 0s 80us/step - loss: 14.6191 - val_loss: 16.8296
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 79us/step - loss: 11.6772 - val_loss: 17.4738
Epoch 218/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.0353 - val_loss: 12.4778
Epoch 219/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.0305 - val_loss: 14.3713
Epoch 220/1000
1052/1052 [==============================] - 0s 78us/step - loss: 11.8809 - val_loss: 13.6023
Epoch 221/1000
1052/1052 [==============================] - 0s 83us/step - loss: 11.9094 - val_loss: 14.9221
Epoch 222/1000
1052/1052 [==============================] - 0s 81us/step - loss: 11.9983 - val_loss: 13.2041
Epoch 223/1000
1052/1052 [==============================] - 0s 81us/step - loss: 11.5469 - val_loss: 13.3953
Epoch 224/1000
1052/1052 [==============================] - 0s 81us/step - loss: 12.0407 - val_loss: 13.0484
Epoch 225/1000
1052/1052 [==============================] - 0s 80us/step - loss: 11.3094 - val_loss: 13.2713
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 84us/step - loss: 10.1586 - val_loss: 12.9773
Epoch 293/1000
1052/1052 [==============================] - 0s 79us/step - loss: 10.4208 - val_loss: 11.0113
Epoch 294/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.9345 - val_loss: 13.9529
Epoch 295/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.9892 - val_loss: 13.7739
Epoch 296/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.9517 - val_loss: 11.3557
Epoch 297/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.9730 - val_loss: 11.3492
Epoch 298/1000
1052/1052 [==============================] - 0s 80us/step - loss: 10.0459 - val_loss: 11.6931
Epoch 299/1000
1052/1052 [==============================] - 0s 76us/step - loss: 10.3020 - val_loss: 11.1410
Epoch 300/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.8603 - val_loss: 11.5562
Epoch 301/1000
1052/1052 [=============

...

1052/1052 [==============================] - 0s 79us/step - loss: 9.0469 - val_loss: 10.0590
Epoch 749/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.0230 - val_loss: 9.6468
Epoch 750/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.3989 - val_loss: 11.3324
Epoch 751/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.4079 - val_loss: 9.5903
Epoch 752/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.1425 - val_loss: 13.1156
Epoch 753/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.4540 - val_loss: 10.7677
Epoch 754/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1861 - val_loss: 11.9812
Epoch 755/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.0247 - val_loss: 10.5013
Epoch 756/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.3539 - val_loss: 11.8602
Epoch 757/1000
1052/1052 [===================

1052/1052 [==============================] - 0s 81us/step - loss: 8.9288 - val_loss: 9.5932
Epoch 825/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.3547 - val_loss: 10.7029
Epoch 826/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2995 - val_loss: 10.2861
Epoch 827/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.0898 - val_loss: 9.6633
Epoch 828/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.7921 - val_loss: 10.3388
Epoch 829/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.5757 - val_loss: 9.9947
Epoch 830/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.8889 - val_loss: 12.8185
Epoch 831/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.5084 - val_loss: 11.6237
Epoch 832/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.3121 - val_loss: 11.1435
Epoch 833/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 82us/step - loss: 9.0974 - val_loss: 9.5079
Epoch 901/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.0800 - val_loss: 9.6094
Epoch 902/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.6173 - val_loss: 9.1419
Epoch 903/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1100 - val_loss: 11.3135
Epoch 904/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.1920 - val_loss: 9.6723
Epoch 905/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.0946 - val_loss: 10.5375
Epoch 906/1000
1052/1052 [==============================] - 0s 74us/step - loss: 8.9458 - val_loss: 11.4338
Epoch 907/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1700 - val_loss: 9.4908
Epoch 908/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.5468 - val_loss: 12.2630
Epoch 909/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 85us/step - loss: 9.2921 - val_loss: 9.5962
Epoch 977/1000
1052/1052 [==============================] - ETA: 0s - loss: 9.636 - 0s 83us/step - loss: 9.2317 - val_loss: 9.7286
Epoch 978/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.0310 - val_loss: 16.2184
Epoch 979/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.9553 - val_loss: 15.9516
Epoch 980/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.3547 - val_loss: 11.4079
Epoch 981/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.6469 - val_loss: 9.5046
Epoch 982/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.9678 - val_loss: 9.8655
Epoch 983/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.2161 - val_loss: 10.9935
Epoch 984/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.8282 - val_loss: 9.4817
Epoch 985/1000
1052/1052

[array([[-5.2773160e-01,  1.9658632e+00, -4.0505700e+00, -3.1017580e+00,
          2.2678111e-01],
        [-9.7718185e-01, -4.4118536e-01,  2.3990002e-01,  2.8758457e-02,
          2.6006827e-01],
        [-1.2281326e+00, -3.7994516e-01, -7.4677682e-01, -2.0932406e-03,
          4.0763730e-01],
        [ 2.0162228e-01, -5.1877890e-02,  1.5202941e-01,  7.0927441e-02,
         -1.9420524e-01],
        [ 3.4209855e-02,  2.8210995e-01, -3.9546540e-01, -2.4980335e+00,
          2.1591848e-01]], dtype=float32),
 array([-0.6113899,  1.2353743, -4.9203124, -4.6428027, -1.9127222],
       dtype=float32),
 array([[-0.7757531 ,  0.16844755,  0.3920855 , -0.53108996,  0.03681013,
          0.44453827,  0.84954506,  0.02229284, -0.6965193 ,  0.12848362],
        [ 0.76073897, -0.77224255, -0.46199623,  0.74619955, -0.25548318,
         -0.41365287,  0.10224669, -0.762868  ,  0.60065097, -0.56195915],
        [ 1.4765929 , -1.0935898 , -2.233196  ,  1.7948748 , -2.0830073 ,
         -1.9415424 , -2

In [26]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_rmsprop_3rd.h5')

In [27]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 616us/step - loss:

274/274 [==============================] - 0s 98us/step - loss: 7.6687 - val_loss: 10.4290
Epoch 70/1000
274/274 [==============================] - 0s 105us/step - loss: 8.1274 - val_loss: 10.9237
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 7.7351 - val_loss: 10.3745
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 7.5991 - val_loss: 10.8048
Epoch 73/1000
274/274 [==============================] - 0s 102us/step - loss: 7.4189 - val_loss: 10.4315
Epoch 74/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3595 - val_loss: 10.5572
Epoch 75/1000
274/274 [==============================] - 0s 102us/step - loss: 7.4090 - val_loss: 10.4770
Epoch 76/1000
274/274 [==============================] - 0s 95us/step - loss: 7.3075 - val_loss: 10.3897
Epoch 77/1000
274/274 [==============================] - 0s 98us/step - loss: 7.2602 - val_loss: 10.3771
Epoch 78/1000
274/274 [==============================] - 0s 95us/s

274/274 [==============================] - 0s 91us/step - loss: 6.7959 - val_loss: 9.0866
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6332 - val_loss: 9.2582
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6539 - val_loss: 9.5572
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7788 - val_loss: 9.0596
Epoch 150/1000
274/274 [==============================] - 0s 87us/step - loss: 6.7439 - val_loss: 9.2353
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8744 - val_loss: 9.2548
Epoch 152/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7259 - val_loss: 9.1214
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5011 - val_loss: 9.1343
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4677 - val_loss: 9.1629
Epoch 155/1000
274/274 [==============================] - 0s 95us/

Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8008 - val_loss: 9.1355
Epoch 225/1000
274/274 [==============================] - 0s 91us/step - loss: 5.9068 - val_loss: 9.1353
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8172 - val_loss: 9.3509
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9248 - val_loss: 9.2963
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8026 - val_loss: 9.2897
Epoch 229/1000
274/274 [==============================] - 0s 96us/step - loss: 5.8325 - val_loss: 9.3979
Epoch 230/1000
274/274 [==============================] - 0s 97us/step - loss: 5.9063 - val_loss: 9.4932
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9383 - val_loss: 9.3471
Epoch 232/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9491 - val_loss: 9.2534
Epoch 233/1000
274/274 [============================

274/274 [==============================] - 0s 109us/step - loss: 5.6728 - val_loss: 9.3860
Epoch 302/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6893 - val_loss: 9.3350
Epoch 303/1000
274/274 [==============================] - ETA: 0s - loss: 4.104 - 0s 107us/step - loss: 5.7594 - val_loss: 9.5427
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8299 - val_loss: 9.3281
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9007 - val_loss: 9.5993
Epoch 306/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8887 - val_loss: 9.0716
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9569 - val_loss: 9.3149
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8127 - val_loss: 9.2562
Epoch 309/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8689 - val_loss: 9.6065
Epoch 310/1000
274/274 [=================

...

274/274 [==============================] - 0s 98us/step - loss: 5.3465 - val_loss: 10.1320
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2471 - val_loss: 10.2638
Epoch 767/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3070 - val_loss: 10.0918
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3428 - val_loss: 9.9823
Epoch 769/1000
274/274 [==============================] - 0s 92us/step - loss: 5.2730 - val_loss: 10.1154
Epoch 770/1000
274/274 [==============================] - 0s 96us/step - loss: 5.3931 - val_loss: 10.0652
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5203 - val_loss: 10.2252
Epoch 772/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9406 - val_loss: 9.8520
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2940 - val_loss: 9.9758
Epoch 774/1000
274/274 [==============================] - 0s 95

274/274 [==============================] - 0s 100us/step - loss: 5.1868 - val_loss: 10.2514
Epoch 843/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2767 - val_loss: 10.1947
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2795 - val_loss: 9.8882
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2147 - val_loss: 10.2109
Epoch 846/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2668 - val_loss: 10.3818
Epoch 847/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4109 - val_loss: 10.2858
Epoch 848/1000
274/274 [==============================] - 0s 101us/step - loss: 5.7405 - val_loss: 10.5659
Epoch 849/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2999 - val_loss: 10.0849
Epoch 850/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2084 - val_loss: 10.5116
Epoch 851/1000
274/274 [==============================] 

Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5141 - val_loss: 10.5604
Epoch 920/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2454 - val_loss: 10.4989
Epoch 921/1000
274/274 [==============================] - 0s 105us/step - loss: 5.2077 - val_loss: 10.1817
Epoch 922/1000
274/274 [==============================] - 0s 99us/step - loss: 5.4504 - val_loss: 10.2902
Epoch 923/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2311 - val_loss: 10.4967
Epoch 924/1000
274/274 [==============================] - 0s 107us/step - loss: 5.1772 - val_loss: 10.2111
Epoch 925/1000
274/274 [==============================] - 0s 100us/step - loss: 5.1074 - val_loss: 10.2704
Epoch 926/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2256 - val_loss: 10.4437
Epoch 927/1000
274/274 [==============================] - 0s 110us/step - loss: 5.1243 - val_loss: 10.4312
Epoch 928/1000
274/274 [================

Epoch 996/1000
274/274 [==============================] - 0s 109us/step - loss: 5.0777 - val_loss: 10.2330
Epoch 997/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0869 - val_loss: 10.5403
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0950 - val_loss: 10.5324
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1936 - val_loss: 10.4793
Epoch 1000/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2417 - val_loss: 10.4618
7.332846851672157


[array([[-1.1355858 ,  0.9510411 , -1.1605895 , -1.9713963 , -2.5113869 ],
        [ 0.43269494,  0.5585832 ,  0.5111336 ,  1.8473979 ,  1.9089156 ],
        [-0.10188517, -0.730202  ,  0.40698844, -0.1939    , -1.8813479 ],
        [-0.96071243, -1.7670507 ,  0.49660054, -2.3780797 , -2.053761  ],
        [-0.4412832 , -0.00680008,  0.74298537,  1.3900087 , -0.01158761],
        [ 1.1548195 ,  1.2759582 , -0.43926668,  1.1732944 ,  0.95736253],
        [ 1.2927997 ,  0.76727563, -1.4178784 , -0.7417816 ,  0.7567058 ]],
       dtype=float32),
 array([-2.9462733, -2.2932255, -1.8466849,  2.0596015, -2.0462449],
       dtype=float32),
 array([[-0.15562584, -0.125597  ,  0.99511504,  0.29056546, -0.5252284 ,
         -0.22972518,  0.7135412 ,  0.6575368 ,  0.4899408 ,  0.60307455],
        [ 0.90416306,  0.72518164, -0.6779449 , -1.3438253 ,  0.37544405,
          1.2401313 , -0.29122335, -0.64681625, -0.51080805, -0.51136935],
        [-0.6325647 ,  0.02354745,  0.36745098, -0.68360955, 

In [28]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_adam_3rd.h5')

In [29]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, sgd, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 507us/step - loss:

274/274 [==============================] - 0s 95us/step - loss: 411.2282 - val_loss: 433.7396
Epoch 68/1000
274/274 [==============================] - 0s 87us/step - loss: 406.8926 - val_loss: 429.1972
Epoch 69/1000
274/274 [==============================] - 0s 95us/step - loss: 402.5758 - val_loss: 424.5938
Epoch 70/1000
274/274 [==============================] - 0s 95us/step - loss: 398.1378 - val_loss: 419.9644
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 393.7175 - val_loss: 415.2250
Epoch 72/1000
274/274 [==============================] - 0s 95us/step - loss: 389.1665 - val_loss: 410.4576
Epoch 73/1000
274/274 [==============================] - 0s 91us/step - loss: 384.5905 - val_loss: 405.6636
Epoch 74/1000
274/274 [==============================] - 0s 98us/step - loss: 380.0165 - val_loss: 400.7815
Epoch 75/1000
274/274 [==============================] - 0s 91us/step - loss: 375.2937 - val_loss: 395.8674
Epoch 76/1000
274/274 [===================

274/274 [==============================] - 0s 84us/step - loss: 72.3212 - val_loss: 78.6427
Epoch 143/1000
274/274 [==============================] - 0s 80us/step - loss: 69.9606 - val_loss: 76.2652
Epoch 144/1000
274/274 [==============================] - 0s 80us/step - loss: 67.7746 - val_loss: 73.9597
Epoch 145/1000
274/274 [==============================] - 0s 84us/step - loss: 65.5813 - val_loss: 71.8173
Epoch 146/1000
274/274 [==============================] - 0s 80us/step - loss: 63.5860 - val_loss: 69.6817
Epoch 147/1000
274/274 [==============================] - 0s 80us/step - loss: 61.6340 - val_loss: 67.6205
Epoch 148/1000
274/274 [==============================] - 0s 91us/step - loss: 59.7124 - val_loss: 65.6642
Epoch 149/1000
274/274 [==============================] - 0s 80us/step - loss: 57.8830 - val_loss: 63.7820
Epoch 150/1000
274/274 [==============================] - 0s 76us/step - loss: 56.0883 - val_loss: 62.0132
Epoch 151/1000
274/274 [============================

274/274 [==============================] - 0s 80us/step - loss: 22.0890 - val_loss: 26.1423
Epoch 219/1000
274/274 [==============================] - 0s 80us/step - loss: 22.0311 - val_loss: 26.0611
Epoch 220/1000
274/274 [==============================] - 0s 84us/step - loss: 21.9750 - val_loss: 25.9811
Epoch 221/1000
274/274 [==============================] - 0s 76us/step - loss: 21.9116 - val_loss: 25.9119
Epoch 222/1000
274/274 [==============================] - 0s 80us/step - loss: 21.8653 - val_loss: 25.8371
Epoch 223/1000
274/274 [==============================] - 0s 80us/step - loss: 21.8087 - val_loss: 25.7697
Epoch 224/1000
274/274 [==============================] - 0s 76us/step - loss: 21.7598 - val_loss: 25.7028
Epoch 225/1000
274/274 [==============================] - 0s 84us/step - loss: 21.7083 - val_loss: 25.6381
Epoch 226/1000
274/274 [==============================] - 0s 80us/step - loss: 21.6619 - val_loss: 25.5758
Epoch 227/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 19.5409 - val_loss: 22.5847
Epoch 295/1000
274/274 [==============================] - 0s 93us/step - loss: 19.5168 - val_loss: 22.5498
Epoch 296/1000
274/274 [==============================] - 0s 87us/step - loss: 19.4907 - val_loss: 22.5170
Epoch 297/1000
274/274 [==============================] - 0s 86us/step - loss: 19.4674 - val_loss: 22.4838
Epoch 298/1000
274/274 [==============================] - 0s 87us/step - loss: 19.4424 - val_loss: 22.4519
Epoch 299/1000
274/274 [==============================] - 0s 85us/step - loss: 19.4188 - val_loss: 22.4174
Epoch 300/1000
274/274 [==============================] - 0s 87us/step - loss: 19.3958 - val_loss: 22.3831
Epoch 301/1000
274/274 [==============================] - 0s 85us/step - loss: 19.3710 - val_loss: 22.3502
Epoch 302/1000
274/274 [==============================] - 0s 91us/step - loss: 19.3493 - val_loss: 22.3192
Epoch 303/1000
274/274 [============================

...

274/274 [==============================] - 0s 80us/step - loss: 13.3547 - val_loss: 15.9892
Epoch 751/1000
274/274 [==============================] - 0s 87us/step - loss: 13.3466 - val_loss: 15.9830
Epoch 752/1000
274/274 [==============================] - 0s 81us/step - loss: 13.3376 - val_loss: 15.9763
Epoch 753/1000
274/274 [==============================] - 0s 80us/step - loss: 13.3291 - val_loss: 15.9712
Epoch 754/1000
274/274 [==============================] - 0s 76us/step - loss: 13.3203 - val_loss: 15.9643
Epoch 755/1000
274/274 [==============================] - 0s 76us/step - loss: 13.3116 - val_loss: 15.9578
Epoch 756/1000
274/274 [==============================] - 0s 80us/step - loss: 13.3044 - val_loss: 15.9504
Epoch 757/1000
274/274 [==============================] - 0s 76us/step - loss: 13.2956 - val_loss: 15.9438
Epoch 758/1000
274/274 [==============================] - 0s 76us/step - loss: 13.2868 - val_loss: 15.9370
Epoch 759/1000
274/274 [============================

274/274 [==============================] - 0s 81us/step - loss: 12.7464 - val_loss: 15.4686
Epoch 827/1000
274/274 [==============================] - 0s 87us/step - loss: 12.7386 - val_loss: 15.4623
Epoch 828/1000
274/274 [==============================] - 0s 76us/step - loss: 12.7316 - val_loss: 15.4553
Epoch 829/1000
274/274 [==============================] - 0s 73us/step - loss: 12.7233 - val_loss: 15.4494
Epoch 830/1000
274/274 [==============================] - 0s 80us/step - loss: 12.7162 - val_loss: 15.4420
Epoch 831/1000
274/274 [==============================] - 0s 80us/step - loss: 12.7088 - val_loss: 15.4349
Epoch 832/1000
274/274 [==============================] - 0s 76us/step - loss: 12.7011 - val_loss: 15.4281
Epoch 833/1000
274/274 [==============================] - 0s 84us/step - loss: 12.6933 - val_loss: 15.4199
Epoch 834/1000
274/274 [==============================] - 0s 80us/step - loss: 12.6860 - val_loss: 15.4113
Epoch 835/1000
274/274 [============================

274/274 [==============================] - 0s 81us/step - loss: 12.1959 - val_loss: 14.9593
Epoch 903/1000
274/274 [==============================] - 0s 87us/step - loss: 12.1893 - val_loss: 14.9531
Epoch 904/1000
274/274 [==============================] - 0s 84us/step - loss: 12.1832 - val_loss: 14.9455
Epoch 905/1000
274/274 [==============================] - 0s 76us/step - loss: 12.1751 - val_loss: 14.9392
Epoch 906/1000
274/274 [==============================] - 0s 80us/step - loss: 12.1691 - val_loss: 14.9326
Epoch 907/1000
274/274 [==============================] - 0s 88us/step - loss: 12.1627 - val_loss: 14.9256
Epoch 908/1000
274/274 [==============================] - 0s 82us/step - loss: 12.1551 - val_loss: 14.9213
Epoch 909/1000
274/274 [==============================] - 0s 84us/step - loss: 12.1479 - val_loss: 14.9172
Epoch 910/1000
274/274 [==============================] - 0s 86us/step - loss: 12.1422 - val_loss: 14.9097
Epoch 911/1000
274/274 [============================

274/274 [==============================] - 0s 80us/step - loss: 11.6986 - val_loss: 14.4940
Epoch 979/1000
274/274 [==============================] - 0s 84us/step - loss: 11.6935 - val_loss: 14.4850
Epoch 980/1000
274/274 [==============================] - 0s 84us/step - loss: 11.6865 - val_loss: 14.4807
Epoch 981/1000
274/274 [==============================] - 0s 80us/step - loss: 11.6812 - val_loss: 14.4745
Epoch 982/1000
274/274 [==============================] - 0s 84us/step - loss: 11.6743 - val_loss: 14.4682
Epoch 983/1000
274/274 [==============================] - 0s 80us/step - loss: 11.6704 - val_loss: 14.4623
Epoch 984/1000
274/274 [==============================] - 0s 80us/step - loss: 11.6621 - val_loss: 14.4556
Epoch 985/1000
274/274 [==============================] - 0s 76us/step - loss: 11.6560 - val_loss: 14.4499
Epoch 986/1000
274/274 [==============================] - 0s 84us/step - loss: 11.6501 - val_loss: 14.4467
Epoch 987/1000
274/274 [============================

[array([[-0.26519018, -0.30969924,  0.31332922,  0.11838782, -0.49619505],
        [-0.29877776,  0.62051237, -0.38825664,  0.13423376, -0.15193045],
        [ 0.0805206 ,  0.2352607 , -0.5637564 ,  0.5941172 ,  0.46963477],
        [-0.48329857, -0.5644614 ,  0.16716547, -0.21568078,  0.05851074],
        [ 0.39602542,  0.10094836, -0.30848545,  0.63181853,  0.6442236 ],
        [ 0.5650914 ,  0.07041802,  0.41007155,  0.26524168, -0.00249722],
        [-0.0894484 ,  0.13705376, -0.01674587, -0.55452013,  0.5080502 ]],
       dtype=float32),
 array([-0.29672837,  1.3756742 ,  0.57918286, -0.6747706 ,  0.10778167],
       dtype=float32),
 array([[-0.35920596,  0.664683  , -0.49307838,  0.06565339,  0.12241128,
          0.8054019 , -0.7028028 ,  0.6012675 , -0.37846255, -0.36771172],
        [-1.1111735 ,  0.3894226 , -0.94691825,  0.71315676, -1.0937616 ,
          0.63859046, -1.2728937 ,  0.24001203, -0.57645357, -0.24197015],
        [-0.58379996,  0.8303262 ,  0.13763455,  0.43417

In [30]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sgd_3rd.h5')

In [31]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, RMSprop, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 611us/step - loss:

274/274 [==============================] - 0s 87us/step - loss: 9.4294 - val_loss: 12.3728
Epoch 69/1000
274/274 [==============================] - 0s 91us/step - loss: 8.6252 - val_loss: 11.5660
Epoch 70/1000
274/274 [==============================] - 0s 84us/step - loss: 8.4552 - val_loss: 10.9516
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 8.1919 - val_loss: 12.0770
Epoch 72/1000
274/274 [==============================] - 0s 87us/step - loss: 9.3007 - val_loss: 11.0262
Epoch 73/1000
274/274 [==============================] - 0s 91us/step - loss: 8.6028 - val_loss: 10.8164
Epoch 74/1000
274/274 [==============================] - 0s 91us/step - loss: 8.4868 - val_loss: 11.5932
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 8.6449 - val_loss: 11.7765
Epoch 76/1000
274/274 [==============================] - 0s 87us/step - loss: 8.2833 - val_loss: 11.4629
Epoch 77/1000
274/274 [==============================] - 0s 91us/step

274/274 [==============================] - 0s 91us/step - loss: 7.7507 - val_loss: 10.8429
Epoch 146/1000
274/274 [==============================] - 0s 91us/step - loss: 8.0943 - val_loss: 11.3923
Epoch 147/1000
274/274 [==============================] - 0s 95us/step - loss: 7.5138 - val_loss: 10.6046
Epoch 148/1000
274/274 [==============================] - 0s 90us/step - loss: 7.8817 - val_loss: 12.7181
Epoch 149/1000
274/274 [==============================] - 0s 95us/step - loss: 7.9875 - val_loss: 10.5390
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 7.6425 - val_loss: 13.0714
Epoch 151/1000
274/274 [==============================] - 0s 98us/step - loss: 8.3561 - val_loss: 10.6659
Epoch 152/1000
274/274 [==============================] - 0s 91us/step - loss: 7.6053 - val_loss: 12.5156
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 8.1425 - val_loss: 11.6573
Epoch 154/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 98us/step - loss: 8.2172 - val_loss: 11.4541
Epoch 223/1000
274/274 [==============================] - 0s 91us/step - loss: 7.3805 - val_loss: 10.7282
Epoch 224/1000
274/274 [==============================] - 0s 100us/step - loss: 7.3350 - val_loss: 11.7117
Epoch 225/1000
274/274 [==============================] - 0s 97us/step - loss: 7.6360 - val_loss: 11.7614
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8903 - val_loss: 12.9476
Epoch 227/1000
274/274 [==============================] - 0s 91us/step - loss: 7.3680 - val_loss: 10.8582
Epoch 228/1000
274/274 [==============================] - 0s 91us/step - loss: 7.4730 - val_loss: 13.2438
Epoch 229/1000
274/274 [==============================] - 0s 91us/step - loss: 8.4032 - val_loss: 11.4554
Epoch 230/1000
274/274 [==============================] - 0s 94us/step - loss: 7.3971 - val_loss: 11.2269
Epoch 231/1000
274/274 [==============================] - 0s

274/274 [==============================] - 0s 106us/step - loss: 8.1567 - val_loss: 11.3022
Epoch 300/1000
274/274 [==============================] - 0s 95us/step - loss: 7.3702 - val_loss: 11.1038
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3471 - val_loss: 12.2247
Epoch 302/1000
274/274 [==============================] - 0s 98us/step - loss: 7.6928 - val_loss: 11.0818
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 7.2303 - val_loss: 10.8775
Epoch 304/1000
274/274 [==============================] - 0s 87us/step - loss: 7.4525 - val_loss: 10.4879
Epoch 305/1000
274/274 [==============================] - 0s 98us/step - loss: 7.8164 - val_loss: 10.6200
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 7.3159 - val_loss: 10.9552
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3207 - val_loss: 10.8278
Epoch 308/1000
274/274 [==============================] - 0

...

274/274 [==============================] - 0s 95us/step - loss: 6.9219 - val_loss: 9.6790
Epoch 762/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7223 - val_loss: 10.0249
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6797 - val_loss: 9.7217
Epoch 764/1000
274/274 [==============================] - 0s 87us/step - loss: 7.5047 - val_loss: 9.8376
Epoch 765/1000
274/274 [==============================] - 0s 92us/step - loss: 6.9027 - val_loss: 9.6336
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6385 - val_loss: 9.6831
Epoch 767/1000
274/274 [==============================] - 0s 95us/step - loss: 6.9459 - val_loss: 10.2659
Epoch 768/1000
274/274 [==============================] - 0s 91us/step - loss: 6.8462 - val_loss: 9.3814
Epoch 769/1000
274/274 [==============================] - 0s 87us/step - loss: 6.8457 - val_loss: 11.4599
Epoch 770/1000
274/274 [==============================] - 0s 98us/s

274/274 [==============================] - 0s 97us/step - loss: 6.5677 - val_loss: 9.8542
Epoch 839/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8366 - val_loss: 9.3841
Epoch 840/1000
274/274 [==============================] - 0s 87us/step - loss: 6.5101 - val_loss: 9.4759
Epoch 841/1000
274/274 [==============================] - 0s 86us/step - loss: 6.6684 - val_loss: 9.8645
Epoch 842/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6618 - val_loss: 11.5784
Epoch 843/1000
274/274 [==============================] - 0s 91us/step - loss: 6.5944 - val_loss: 10.4323
Epoch 844/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7684 - val_loss: 9.4510
Epoch 845/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5950 - val_loss: 10.1392
Epoch 846/1000
274/274 [==============================] - 0s 84us/step - loss: 6.5910 - val_loss: 10.3716
Epoch 847/1000
274/274 [==============================] - 0s 88us/

274/274 [==============================] - 0s 95us/step - loss: 6.4187 - val_loss: 9.3205
Epoch 916/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6642 - val_loss: 11.3639
Epoch 917/1000
274/274 [==============================] - 0s 87us/step - loss: 7.0117 - val_loss: 9.7159
Epoch 918/1000
274/274 [==============================] - 0s 91us/step - loss: 6.4571 - val_loss: 10.5677
Epoch 919/1000
274/274 [==============================] - 0s 95us/step - loss: 7.0116 - val_loss: 9.5736
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3258 - val_loss: 11.8445
Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6947 - val_loss: 9.1771
Epoch 922/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5756 - val_loss: 12.6802
Epoch 923/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8097 - val_loss: 11.0814
Epoch 924/1000
274/274 [==============================] - 0s 109

274/274 [==============================] - 0s 106us/step - loss: 6.7180 - val_loss: 9.0474
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4683 - val_loss: 9.4997
Epoch 994/1000
274/274 [==============================] - 0s 94us/step - loss: 6.3697 - val_loss: 10.2518
Epoch 995/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7303 - val_loss: 9.0426
Epoch 996/1000
274/274 [==============================] - 0s 91us/step - loss: 6.8842 - val_loss: 9.2125
Epoch 997/1000
274/274 [==============================] - 0s 91us/step - loss: 6.7252 - val_loss: 9.1173
Epoch 998/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6397 - val_loss: 9.3963
Epoch 999/1000
274/274 [==============================] - 0s 96us/step - loss: 6.5534 - val_loss: 10.4593
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9526 - val_loss: 9.3224
8.418632361848475


[array([[-2.9194338 ,  1.1333501 ,  1.633902  ,  1.2145857 ,  0.21644014],
        [-1.6962245 ,  1.0549319 , -0.32198128, -0.9354039 , -0.38935444],
        [ 0.831562  ,  0.9823663 ,  0.18948533, -0.5177159 ,  0.02019707],
        [ 0.32788152, -0.4750337 , -0.75743586,  0.36078483,  0.673329  ],
        [-0.13353384,  0.21440934, -0.0337398 , -1.0063103 , -0.2418501 ],
        [-1.9717413 ,  0.6557197 ,  0.19731179,  0.48653534, -0.26119944],
        [ 0.11370736, -0.09005906, -0.3879497 ,  1.4666969 , -0.14133136]],
       dtype=float32),
 array([-2.2606797 ,  1.1657685 ,  2.362478  ,  2.1019611 , -0.01859286],
       dtype=float32),
 array([[ 0.693732  ,  0.07137936,  0.42482084, -0.605629  ,  0.7224768 ,
         -0.7093322 , -0.74295974, -0.18425886, -0.91097474, -0.06839804],
        [ 0.91033924, -1.0121555 ,  0.9812546 , -0.12327425,  0.70182806,
         -0.8012934 , -0.8114219 ,  0.66358215, -0.86572164,  0.38702586],
        [-0.8012143 ,  0.75383145, -0.68374926,  1.28516

In [32]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_rmsprop_3rd.h5')

In [33]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 105us/step - loss: 0.0058 - val_loss: 0.0045
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0056
Epoch 71/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0059 - val_loss: 0.0046
Epoch 72/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0063 - val_loss: 0.0064
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0043
Epoch 74/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0058 - val_loss: 0.0079
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0072 - val_loss: 0.0049
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0065 - val_loss: 0.0051
Epoch 78/1000
143/143 [==============================] - 0s 112us/step

Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0054
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0040
Epoch 149/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0044 - val_loss: 0.0046
Epoch 150/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0043
Epoch 151/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0044 - val_loss: 0.0058
Epoch 152/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0046 - val_loss: 0.0040
Epoch 153/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0046 - val_loss: 0.0045
Epoch 154/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 155/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0044
Epoch 156/1000
143/143 [=======================

143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0044
Epoch 225/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0047 - val_loss: 0.0042
Epoch 226/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0041 - val_loss: 0.0071
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0040
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0042
Epoch 229/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 230/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0039 - val_loss: 0.0039
Epoch 231/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0044 - val_loss: 0.0095
Epoch 232/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0059 - val_loss: 0.0050
Epoch 233/1000
143/143 [==============================] - 0s 105us

143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0039
Epoch 302/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0036 - val_loss: 0.0037
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0038 - val_loss: 0.0048
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0037
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0039
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0043
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0034 - val_loss: 0.0041
Epoch 308/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0047
Epoch 309/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0046
Epoch 310/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0028
Epoch 764/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0024 - val_loss: 0.0026
Epoch 765/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0039
Epoch 766/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0028
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0042
Epoch 769/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0029
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0027
Epoch 771/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0040
Epoch 772/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0030
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0060
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0029
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0029
Epoch 845/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0021 - val_loss: 0.0045
Epoch 846/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0024 - val_loss: 0.0028
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0034
Epoch 848/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0024 - val_loss: 0.0040
Epoch 849/1000
143/143 [==============================] - 0s 98

143/143 [==============================] - 0s 110us/step - loss: 0.0022 - val_loss: 0.0033
Epoch 918/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0029
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0031
Epoch 921/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0027
Epoch 922/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0023 - val_loss: 0.0066
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0030 - val_loss: 0.0030
Epoch 924/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0038 - val_loss: 0.0030
Epoch 925/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0065
Epoch 926/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 995/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0024 - val_loss: 0.0031
Epoch 996/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0022 - val_loss: 0.0039
Epoch 997/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0023 - val_loss: 0.0035
Epoch 998/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0022 - val_loss: 0.0029
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0020 - val_loss: 0.0041
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0032
0.005973733030259609


[array([[-0.74030954,  0.3146958 ,  0.7497738 ,  0.55560756, -0.3412879 ],
        [ 0.57891   ,  0.06820694,  0.40552258, -0.3317556 , -0.07173084],
        [ 0.78854215, -0.18321228,  0.04109332, -0.36616966, -0.781797  ],
        [ 0.19663054,  0.06569327,  0.5075768 , -0.08712752,  0.27043134],
        [ 0.25546685,  0.09687472,  0.2534934 , -0.03519319,  0.26828328],
        [-0.73063266, -0.5608398 ,  0.34035873, -0.06075739, -0.6558311 ],
        [ 0.2619733 ,  0.285457  ,  0.36613446,  0.00537155, -0.27280027],
        [-0.2415304 ,  0.6199488 ,  0.4111679 ,  0.54069245, -0.19252339],
        [-0.4500729 , -0.92395854,  0.19867781, -0.19394769, -0.1515075 ],
        [ 0.62895   ,  1.7142737 , -0.18603234,  0.42516747, -0.495509  ],
        [ 0.14318705,  0.6520148 ,  0.09665535, -0.36531088, -0.32650274],
        [ 0.35333183,  1.1119828 ,  0.05291332,  0.6647812 , -0.4951954 ],
        [ 0.6712026 ,  0.724882  ,  0.33256117, -0.2680109 ,  0.07979652],
        [-2.0637224 , -0.

In [34]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_adam_3rd.h5')

In [35]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, sgd, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 97us/step - loss: 0.0742 - val_loss: 0.0562
Epoch 70/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0742 - val_loss: 0.0562
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0741 - val_loss: 0.0562
Epoch 72/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0741 - val_loss: 0.0562
Epoch 73/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0741 - val_loss: 0.0562
Epoch 74/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0741 - val_loss: 0.0562
Epoch 75/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0741 - val_loss: 0.0562
Epoch 76/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0740 - val_loss: 0.0562
Epoch 77/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0740 - val_loss: 0.0562
Epoch 78/1000
143/143 [==============================] - 0s 77us/step - loss: 

143/143 [==============================] - 0s 98us/step - loss: 0.0725 - val_loss: 0.0556
Epoch 148/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0725 - val_loss: 0.0556
Epoch 149/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0725 - val_loss: 0.0556
Epoch 150/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0724 - val_loss: 0.0556
Epoch 151/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0724 - val_loss: 0.0556
Epoch 152/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0724 - val_loss: 0.0556
Epoch 153/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0724 - val_loss: 0.0556
Epoch 154/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0724 - val_loss: 0.0556
Epoch 155/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0723 - val_loss: 0.0556
Epoch 156/1000
143/143 [==============================] - 0s 84us/step

143/143 [==============================] - 0s 98us/step - loss: 0.0710 - val_loss: 0.0551
Epoch 226/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0710 - val_loss: 0.0551
Epoch 227/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0710 - val_loss: 0.0551
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0710 - val_loss: 0.0551
Epoch 229/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0709 - val_loss: 0.0551
Epoch 230/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0709 - val_loss: 0.0551
Epoch 231/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0709 - val_loss: 0.0551
Epoch 232/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0709 - val_loss: 0.0551
Epoch 233/1000
143/143 [==============================] - 0s 90us/step - loss: 0.0709 - val_loss: 0.0551
Epoch 234/1000
143/143 [==============================] - 0s 91us/ste

143/143 [==============================] - 0s 98us/step - loss: 0.0697 - val_loss: 0.0547
Epoch 304/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0697 - val_loss: 0.0547
Epoch 305/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0697 - val_loss: 0.0547
Epoch 306/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0697 - val_loss: 0.0547
Epoch 307/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0696 - val_loss: 0.0547
Epoch 308/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0696 - val_loss: 0.0547
Epoch 309/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0696 - val_loss: 0.0547
Epoch 310/1000
143/143 [==============================] - 0s 90us/step - loss: 0.0696 - val_loss: 0.0547
Epoch 311/1000
143/143 [==============================] - 0s 90us/step - loss: 0.0696 - val_loss: 0.0547
Epoch 312/1000
143/143 [==============================] - 0s 77us/ste

...

143/143 [==============================] - 0s 91us/step - loss: 0.0648 - val_loss: 0.0528
Epoch 694/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0648 - val_loss: 0.0528
Epoch 695/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0648 - val_loss: 0.0528
Epoch 696/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0648 - val_loss: 0.0528
Epoch 697/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0648 - val_loss: 0.0528
Epoch 698/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0647 - val_loss: 0.0528
Epoch 699/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0647 - val_loss: 0.0528
Epoch 700/1000
143/143 [==============================] - 0s 90us/step - loss: 0.0647 - val_loss: 0.0528
Epoch 701/1000
143/143 [==============================] - 0s 90us/step - loss: 0.0647 - val_loss: 0.0528
Epoch 702/1000
143/143 [==============================] - 0s 84us/step

143/143 [==============================] - 0s 91us/step - loss: 0.0640 - val_loss: 0.0525
Epoch 772/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0640 - val_loss: 0.0525
Epoch 773/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0640 - val_loss: 0.0525
Epoch 774/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0640 - val_loss: 0.0525
Epoch 775/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0640 - val_loss: 0.0525
Epoch 776/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0640 - val_loss: 0.0524
Epoch 777/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0640 - val_loss: 0.0524
Epoch 778/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0640 - val_loss: 0.0524
Epoch 779/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0640 - val_loss: 0.0524
Epoch 780/1000
143/143 [==============================] - 0s 98us/step

143/143 [==============================] - 0s 98us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 849/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 850/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 851/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 852/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 853/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 854/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 855/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0633 - val_loss: 0.0521
Epoch 856/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0632 - val_loss: 0.0521
Epoch 857/1000
143/143 [==============================] - 0s 84us/step

Epoch 926/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0626 - val_loss: 0.0518
Epoch 927/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 928/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 929/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 930/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 931/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 932/1000
143/143 [==============================] - 0s 76us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 933/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 934/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0626 - val_loss: 0.0517
Epoch 935/1000
143/143 [==============================

[array([[-0.32830662,  0.2967176 , -0.15410584,  0.12707569,  0.42532948],
        [ 0.42293137,  0.32847103,  0.36824897,  0.2350426 ,  0.09040204],
        [ 0.33595762, -0.05446576, -0.27066475, -0.21815567,  0.07014158],
        [-0.17458367,  0.2179738 , -0.43622857, -0.00713356,  0.24799232],
        [ 0.4282931 ,  0.3506481 ,  0.3720785 ,  0.14501937, -0.37635833],
        [ 0.29348272, -0.2959374 ,  0.46123862, -0.17226104, -0.23436987],
        [-0.22771536, -0.15250465,  0.36093733, -0.10753572,  0.12690161],
        [ 0.28682417, -0.11349607,  0.11926742, -0.430047  ,  0.34092727],
        [-0.18491907,  0.44185424, -0.00642083, -0.08693419,  0.04187988],
        [-0.33756962, -0.0199553 ,  0.41007975,  0.46810055, -0.42826802],
        [ 0.3661651 , -0.21436523, -0.12946492,  0.02425014, -0.2741015 ],
        [ 0.29413012, -0.16408733, -0.2197242 , -0.05383405,  0.13959703],
        [ 0.09571302, -0.01731401,  0.4350091 , -0.3648045 , -0.2990119 ],
        [ 0.07844742,  0.

In [36]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sgd_3rd.h5')

In [37]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, RMSprop, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 98us/step - loss: 0.0162 - val_loss: 0.0108
Epoch 70/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0125 - val_loss: 0.0098
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0141 - val_loss: 0.0113
Epoch 72/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0136 - val_loss: 0.0110
Epoch 73/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0111 - val_loss: 0.0119
Epoch 74/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0101 - val_loss: 0.0071
Epoch 75/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0103 - val_loss: 0.0064
Epoch 76/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0126 - val_loss: 0.0184
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0114 - val_loss: 0.0065
Epoch 78/1000
143/143 [==============================] - 0s 91us/step - loss: 

143/143 [==============================] - 0s 105us/step - loss: 0.0127 - val_loss: 0.0072
Epoch 148/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0073 - val_loss: 0.0057
Epoch 149/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0080 - val_loss: 0.0075
Epoch 150/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0083 - val_loss: 0.0044
Epoch 151/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0074 - val_loss: 0.0051
Epoch 152/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0103 - val_loss: 0.0060
Epoch 153/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0082 - val_loss: 0.0050
Epoch 154/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0085 - val_loss: 0.0075
Epoch 155/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0116 - val_loss: 0.0050
Epoch 156/1000
143/143 [==============================] - 0s 98us

Epoch 225/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0088 - val_loss: 0.0059
Epoch 226/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0066 - val_loss: 0.0047
Epoch 227/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0073 - val_loss: 0.0049
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0073 - val_loss: 0.0053
Epoch 229/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0066 - val_loss: 0.0051
Epoch 230/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0091 - val_loss: 0.0071
Epoch 231/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0067 - val_loss: 0.0045
Epoch 232/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0064 - val_loss: 0.0077
Epoch 233/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0082 - val_loss: 0.0047
Epoch 234/1000
143/143 [=========================

143/143 [==============================] - 0s 90us/step - loss: 0.0060 - val_loss: 0.0051
Epoch 303/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0065 - val_loss: 0.0089
Epoch 304/1000
143/143 [==============================] - 0s 93us/step - loss: 0.0075 - val_loss: 0.0055
Epoch 305/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0069 - val_loss: 0.0044
Epoch 306/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0065 - val_loss: 0.0039
Epoch 307/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0068 - val_loss: 0.0041
Epoch 308/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0045
Epoch 309/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0077 - val_loss: 0.0046
Epoch 310/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0059 - val_loss: 0.0051
Epoch 311/1000
143/143 [==============================] - 0s 105us/s

...

Epoch 767/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0035
Epoch 768/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0058 - val_loss: 0.0039
Epoch 769/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0034 - val_loss: 0.0028
Epoch 770/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0033 - val_loss: 0.0037
Epoch 771/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0051 - val_loss: 0.0020
Epoch 772/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0028
Epoch 773/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0044
Epoch 774/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0044 - val_loss: 0.0073
Epoch 775/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0047 - val_loss: 0.0023
Epoch 776/1000
143/143 [===========================

143/143 [==============================] - 0s 105us/step - loss: 0.0046 - val_loss: 0.0036
Epoch 844/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0039 - val_loss: 0.0038
Epoch 845/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0039 - val_loss: 0.0031
Epoch 846/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0040 - val_loss: 0.0020
Epoch 847/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0032 - val_loss: 0.0021
Epoch 848/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0045 - val_loss: 0.0025
Epoch 849/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0045 - val_loss: 0.0022
Epoch 850/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0036 - val_loss: 0.0020
Epoch 851/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0031 - val_loss: 0.0021
Epoch 852/1000
143/143 [==============================] - 0s 98us/

Epoch 921/1000
143/143 [==============================] - ETA: 0s - loss: 0.003 - 0s 98us/step - loss: 0.0038 - val_loss: 0.0028
Epoch 922/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 923/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0037 - val_loss: 0.0036
Epoch 924/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0029 - val_loss: 0.0021
Epoch 925/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0029 - val_loss: 0.0035
Epoch 926/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0045 - val_loss: 0.0048
Epoch 927/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0047 - val_loss: 0.0020
Epoch 928/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0028 - val_loss: 0.0044
Epoch 929/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0031 - val_loss: 0.0026
Epoch 930/1000
143/143 [=====

143/143 [==============================] - 0s 91us/step - loss: 0.0029 - val_loss: 0.0024
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0054 - val_loss: 0.0025
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0022
0.007098103407770395


[array([[ 0.5582802 , -0.04097692, -0.396288  ,  1.0269827 ,  0.04957781],
        [ 0.05195243, -1.020368  ,  0.8871208 , -0.6734438 , -0.34357658],
        [ 0.6671675 ,  0.21326268, -0.2777804 , -0.76889265,  0.03472825],
        [ 0.3636443 , -0.6600855 , -0.30317295,  0.24650797, -0.209651  ],
        [ 0.09387331,  0.4269042 ,  0.9981767 , -0.24441618, -0.07885956],
        [ 0.07322767, -0.1910683 ,  0.5353461 ,  0.6229598 ,  0.04038649],
        [ 0.5303899 , -1.6075166 ,  0.673626  , -0.4315118 ,  0.11180747],
        [-0.03200595, -0.4392968 , -0.46041298,  0.317933  , -0.12328755],
        [ 0.43916726, -0.36721978,  0.3815556 , -0.15650061,  0.00657681],
        [ 0.14763746, -0.71152484, -0.47883058,  0.5012954 , -0.03977246],
        [-0.48034444, -0.01123167, -0.10017574, -0.49393874, -0.01887495],
        [-2.507587  ,  1.23152   ,  0.17993413, -0.4119466 ,  0.26920363],
        [-0.6396042 ,  0.56893194, -0.9171665 , -0.37330353,  0.33030975],
        [-3.8663263 , -0.

In [38]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_rmsprop_3rd.h5')

In [39]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 36.189

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 0.0028
Epoch 72/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0033 - val_loss: 0.0026
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0029
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0034
Epoch 76/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0045 - val_loss: 0.0040
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0048
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0045 - val_loss: 0.0057
Epoch 79/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0044 - val_loss: 0.0063
Epoch 80/1000
32/32 [==============================] - 0s 125us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0010 - val_loss: 0.0011
Epoch 154/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0010 - val_loss: 0.0011
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0011
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0011
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0010 - val_loss: 0.0010
Epoch 158/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0010
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0010
Epoch 160/1000
32/32 [==============================] - 0s 124us

Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5090e-04 - val_loss: 5.4269e-04
Epoch 226/1000
32/32 [==============================] - 0s 124us/step - loss: 6.4691e-04 - val_loss: 5.3793e-04
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 6.4296e-04 - val_loss: 5.3318e-04
Epoch 228/1000
32/32 [==============================] - 0s 125us/step - loss: 6.3903e-04 - val_loss: 5.2847e-04
Epoch 229/1000
32/32 [==============================] - 0s 155us/step - loss: 6.3513e-04 - val_loss: 5.2380e-04
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3127e-04 - val_loss: 5.1919e-04
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2743e-04 - val_loss: 5.1466e-04
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2363e-04 - val_loss: 5.1019e-04
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1984e-04 - val_loss: 5.0

Epoch 299/1000
32/32 [==============================] - 0s 155us/step - loss: 4.2380e-04 - val_loss: 2.8597e-04
Epoch 300/1000
32/32 [==============================] - 0s 124us/step - loss: 4.2149e-04 - val_loss: 2.8351e-04
Epoch 301/1000
32/32 [==============================] - 0s 155us/step - loss: 4.1920e-04 - val_loss: 2.8108e-04
Epoch 302/1000
32/32 [==============================] - 0s 124us/step - loss: 4.1691e-04 - val_loss: 2.7866e-04
Epoch 303/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1464e-04 - val_loss: 2.7627e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1240e-04 - val_loss: 2.7389e-04
Epoch 305/1000
32/32 [==============================] - 0s 157us/step - loss: 4.1016e-04 - val_loss: 2.7155e-04
Epoch 306/1000
32/32 [==============================] - 0s 155us/step - loss: 4.0794e-04 - val_loss: 2.6922e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0573e-04 - val_loss: 2.6

...

Epoch 743/1000
32/32 [==============================] - 0s 124us/step - loss: 6.9726e-05 - val_loss: 1.6358e-06
Epoch 744/1000
32/32 [==============================] - 0s 125us/step - loss: 6.9533e-05 - val_loss: 1.6162e-06
Epoch 745/1000
32/32 [==============================] - 0s 156us/step - loss: 6.9338e-05 - val_loss: 1.5976e-06
Epoch 746/1000
32/32 [==============================] - 0s 124us/step - loss: 6.9145e-05 - val_loss: 1.5787e-06
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8954e-05 - val_loss: 1.5607e-06
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8761e-05 - val_loss: 1.5432e-06
Epoch 749/1000
32/32 [==============================] - 0s 124us/step - loss: 6.8571e-05 - val_loss: 1.5265e-06
Epoch 750/1000
32/32 [==============================] - 0s 155us/step - loss: 6.8379e-05 - val_loss: 1.5101e-06
Epoch 751/1000
32/32 [==============================] - 0s 125us/step - loss: 6.8194e-05 - val_loss: 1.4

Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7447e-05 - val_loss: 1.4646e-06
Epoch 818/1000
32/32 [==============================] - 0s 124us/step - loss: 5.7309e-05 - val_loss: 1.4757e-06
Epoch 819/1000
32/32 [==============================] - 0s 125us/step - loss: 5.7171e-05 - val_loss: 1.4882e-06
Epoch 820/1000
32/32 [==============================] - 0s 124us/step - loss: 5.7033e-05 - val_loss: 1.4999e-06
Epoch 821/1000
32/32 [==============================] - 0s 125us/step - loss: 5.6892e-05 - val_loss: 1.5132e-06
Epoch 822/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6753e-05 - val_loss: 1.5253e-06
Epoch 823/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6619e-05 - val_loss: 1.5389e-06
Epoch 824/1000
32/32 [==============================] - 0s 125us/step - loss: 5.6483e-05 - val_loss: 1.5514e-06
Epoch 825/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6345e-05 - val_loss: 1.5

Epoch 891/1000
32/32 [==============================] - 0s 125us/step - loss: 4.8545e-05 - val_loss: 2.9080e-06
Epoch 892/1000
32/32 [==============================] - 0s 125us/step - loss: 4.8441e-05 - val_loss: 2.9335e-06
Epoch 893/1000
32/32 [==============================] - 0s 155us/step - loss: 4.8342e-05 - val_loss: 2.9592e-06
Epoch 894/1000
32/32 [==============================] - 0s 125us/step - loss: 4.8237e-05 - val_loss: 2.9837e-06
Epoch 895/1000
32/32 [==============================] - 0s 125us/step - loss: 4.8137e-05 - val_loss: 3.0090e-06
Epoch 896/1000
32/32 [==============================] - 0s 155us/step - loss: 4.8034e-05 - val_loss: 3.0344e-06
Epoch 897/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7936e-05 - val_loss: 3.0611e-06
Epoch 898/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7834e-05 - val_loss: 3.0862e-06
Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7735e-05 - val_loss: 3.1

Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1925e-05 - val_loss: 4.9441e-06
Epoch 966/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1844e-05 - val_loss: 4.9753e-06
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1769e-05 - val_loss: 5.0023e-06
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1693e-05 - val_loss: 5.0307e-06
Epoch 969/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1615e-05 - val_loss: 5.0611e-06
Epoch 970/1000
32/32 [==============================] - 0s 124us/step - loss: 4.1539e-05 - val_loss: 5.0906e-06
Epoch 971/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1464e-05 - val_loss: 5.1170e-06
Epoch 972/1000
32/32 [==============================] - 0s 124us/step - loss: 4.1387e-05 - val_loss: 5.1458e-06
Epoch 973/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1311e-05 - val_loss: 5.1

[array([[-1.06979   , -0.7961142 ,  0.64475465, -0.9167824 , -0.7710856 ],
        [-0.00570277,  0.1451112 ,  0.06422856,  0.00160196,  0.18474798],
        [-0.40886483,  0.8349889 ,  1.4544225 , -1.2489926 , -0.7169367 ]],
       dtype=float32),
 array([ 0.52216   , -0.5162961 ,  0.8141388 , -0.61387265,  0.70759046],
       dtype=float32),
 array([[ 0.13249971, -0.02749045, -0.06980216, -0.43331832,  0.29314613,
         -0.41388956,  0.18524103,  0.33450633,  0.38046607, -0.32907957],
        [-0.3927727 ,  0.04278682,  0.3939044 , -0.19297487,  0.00772749,
          0.48491824,  0.12695453, -0.38771734,  0.12690748, -0.0568864 ],
        [ 0.31674573, -0.22700183, -0.43078703, -0.26720774, -0.4426306 ,
         -0.03089639,  0.21196915,  0.7741325 ,  0.11010344, -0.06774402],
        [ 0.4648844 ,  0.139644  ,  0.09781236, -0.6226803 , -0.34589076,
         -0.07471814,  0.24983853, -0.14483215,  0.15308753, -0.42063817],
        [ 0.744189  ,  0.09776539, -0.39871383,  0.1119473

In [40]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_adam_3rd.h5')

In [41]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, sgd, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 5ms/step - loss: 36.432

Epoch 70/1000
32/32 [==============================] - 0s 125us/step - loss: 35.9016 - val_loss: 42.2815
Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 35.8929 - val_loss: 42.2718
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 35.8843 - val_loss: 42.2621
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 35.8757 - val_loss: 42.2525
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 35.8670 - val_loss: 42.2428
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 35.8584 - val_loss: 42.2331
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 35.8498 - val_loss: 42.2234
Epoch 77/1000
32/32 [==============================] - 0s 94us/step - loss: 35.8411 - val_loss: 42.2137
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 35.8325 - val_loss: 42.2041
Epoch 79/1000
32/32 [==============================] - 0

Epoch 148/1000
32/32 [==============================] - 0s 124us/step - loss: 35.2344 - val_loss: 41.5344
Epoch 149/1000
32/32 [==============================] - 0s 124us/step - loss: 35.2259 - val_loss: 41.5249
Epoch 150/1000
32/32 [==============================] - 0s 125us/step - loss: 35.2175 - val_loss: 41.5155
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 35.2090 - val_loss: 41.5060
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 35.2006 - val_loss: 41.4966
Epoch 153/1000
32/32 [==============================] - 0s 133us/step - loss: 35.1921 - val_loss: 41.4871
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 35.1837 - val_loss: 41.4777
Epoch 155/1000
32/32 [==============================] - 0s 124us/step - loss: 35.1752 - val_loss: 41.4682
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 35.1668 - val_loss: 41.4588
Epoch 157/1000
32/32 [========================

Epoch 226/1000
32/32 [==============================] - 0s 94us/step - loss: 34.5812 - val_loss: 40.8053
Epoch 227/1000
32/32 [==============================] - 0s 94us/step - loss: 34.5729 - val_loss: 40.7960
Epoch 228/1000
32/32 [==============================] - 0s 94us/step - loss: 34.5646 - val_loss: 40.7868
Epoch 229/1000
32/32 [==============================] - 0s 93us/step - loss: 34.5563 - val_loss: 40.7776
Epoch 230/1000
32/32 [==============================] - 0s 94us/step - loss: 34.5481 - val_loss: 40.7683
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 34.5398 - val_loss: 40.7591
Epoch 232/1000
32/32 [==============================] - 0s 93us/step - loss: 34.5315 - val_loss: 40.7499
Epoch 233/1000
32/32 [==============================] - 0s 126us/step - loss: 34.5232 - val_loss: 40.7407
Epoch 234/1000
32/32 [==============================] - 0s 94us/step - loss: 34.5150 - val_loss: 40.7315
Epoch 235/1000
32/32 [==============================]

Epoch 304/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9409 - val_loss: 40.0929
Epoch 305/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9328 - val_loss: 40.0838
Epoch 306/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9247 - val_loss: 40.0748
Epoch 307/1000
32/32 [==============================] - 0s 124us/step - loss: 33.9166 - val_loss: 40.0658
Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9084 - val_loss: 40.0568
Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9003 - val_loss: 40.0477
Epoch 310/1000
32/32 [==============================] - 0s 93us/step - loss: 33.8922 - val_loss: 40.0387
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 33.8841 - val_loss: 40.0297
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 33.8760 - val_loss: 40.0207
Epoch 313/1000
32/32 [=========================

...

Epoch 694/1000
32/32 [==============================] - 0s 124us/step - loss: 30.9029 - val_loss: 36.7425
Epoch 695/1000
32/32 [==============================] - 0s 125us/step - loss: 30.8954 - val_loss: 36.7343
Epoch 696/1000
32/32 [==============================] - 0s 124us/step - loss: 30.8879 - val_loss: 36.7261
Epoch 697/1000
32/32 [==============================] - 0s 125us/step - loss: 30.8804 - val_loss: 36.7179
Epoch 698/1000
32/32 [==============================] - 0s 94us/step - loss: 30.8730 - val_loss: 36.7097
Epoch 699/1000
32/32 [==============================] - 0s 125us/step - loss: 30.8655 - val_loss: 36.7015
Epoch 700/1000
32/32 [==============================] - 0s 125us/step - loss: 30.8580 - val_loss: 36.6933
Epoch 701/1000
32/32 [==============================] - 0s 94us/step - loss: 30.8505 - val_loss: 36.6851
Epoch 702/1000
32/32 [==============================] - 0s 94us/step - loss: 30.8430 - val_loss: 36.6769
Epoch 703/1000
32/32 [===========================

Epoch 772/1000
32/32 [==============================] - 0s 125us/step - loss: 30.3227 - val_loss: 36.1080
Epoch 773/1000
32/32 [==============================] - 0s 125us/step - loss: 30.3154 - val_loss: 36.1000
Epoch 774/1000
32/32 [==============================] - 0s 156us/step - loss: 30.3080 - val_loss: 36.0919
Epoch 775/1000
32/32 [==============================] - 0s 125us/step - loss: 30.3006 - val_loss: 36.0838
Epoch 776/1000
32/32 [==============================] - 0s 93us/step - loss: 30.2932 - val_loss: 36.0758
Epoch 777/1000
32/32 [==============================] - 0s 125us/step - loss: 30.2858 - val_loss: 36.0677
Epoch 778/1000
32/32 [==============================] - 0s 125us/step - loss: 30.2784 - val_loss: 36.0596
Epoch 779/1000
32/32 [==============================] - 0s 156us/step - loss: 30.2711 - val_loss: 36.0516
Epoch 780/1000
32/32 [==============================] - 0s 125us/step - loss: 30.2637 - val_loss: 36.0435
Epoch 781/1000
32/32 [=========================

Epoch 850/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7503 - val_loss: 35.4836
Epoch 851/1000
32/32 [==============================] - 0s 93us/step - loss: 29.7430 - val_loss: 35.4756
Epoch 852/1000
32/32 [==============================] - 0s 124us/step - loss: 29.7357 - val_loss: 35.4677
Epoch 853/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7284 - val_loss: 35.4598
Epoch 854/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7212 - val_loss: 35.4518
Epoch 855/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7139 - val_loss: 35.4439
Epoch 856/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7066 - val_loss: 35.4359
Epoch 857/1000
32/32 [==============================] - 0s 125us/step - loss: 29.6993 - val_loss: 35.4280
Epoch 858/1000
32/32 [==============================] - 0s 125us/step - loss: 29.6920 - val_loss: 35.4201
Epoch 859/1000
32/32 [=========================

Epoch 928/1000
32/32 [==============================] - 0s 93us/step - loss: 29.1851 - val_loss: 34.8685
Epoch 929/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1779 - val_loss: 34.8606
Epoch 930/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1707 - val_loss: 34.8528
Epoch 931/1000
32/32 [==============================] - 0s 94us/step - loss: 29.1635 - val_loss: 34.8450
Epoch 932/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1563 - val_loss: 34.8372
Epoch 933/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1491 - val_loss: 34.8293
Epoch 934/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1419 - val_loss: 34.8215
Epoch 935/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1347 - val_loss: 34.8137
Epoch 936/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1275 - val_loss: 34.8059
Epoch 937/1000
32/32 [==========================

[array([[-0.43328637,  0.61841697,  0.4175151 , -0.54024446, -0.30533135],
        [ 0.32229295, -0.80676603, -0.07286493, -0.58304614,  0.46756256],
        [ 0.37517905, -0.5873628 , -0.08248745,  0.7249734 ,  0.08239289]],
       dtype=float32),
 array([-0.00223206,  0.07089156,  0.04177709, -0.11450855,  0.05202231],
       dtype=float32),
 array([[-0.20162421, -0.50155556,  0.41630778, -0.21395391,  0.06132128,
          0.5246591 , -0.04366466,  0.38296893,  0.12880528,  0.3228862 ],
        [ 0.3694427 ,  0.23171738, -0.2943108 ,  0.33254442,  0.3371436 ,
         -0.12001829,  0.606845  ,  0.43571863, -0.6092993 ,  0.10571722],
        [-0.21710646, -0.4683766 , -0.03660404,  0.29019648,  0.205823  ,
          0.1370113 ,  0.14389914, -0.46675196,  0.60453   , -0.02389203],
        [-0.45574063,  0.07799549,  0.22872703,  0.2731684 , -0.05105175,
         -0.5864233 , -0.2467764 , -0.21987729,  0.22584309, -0.3252195 ],
        [-0.59373724, -0.6211349 , -0.53187734, -0.5176931

In [42]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sgd_3rd.h5')

In [43]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, RMSprop, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 35.983

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0614 - val_loss: 0.0350
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0606 - val_loss: 0.0794
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0612 - val_loss: 0.0319
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0670 - val_loss: 0.0990
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0765 - val_loss: 0.0399
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0937 - val_loss: 0.1230
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0975 - val_loss: 0.0499
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0984 - val_loss: 0.1075
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0814 - val_loss: 0.0451
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0234 - val_loss: 0.0069
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0234 - val_loss: 0.0188
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0273 - val_loss: 0.0192
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0376 - val_loss: 0.0352
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0459 - val_loss: 0.0399
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0542 - val_loss: 0.0455
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0491 - val_loss: 0.0471
Epoch 158/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0476 - val_loss: 0.0412
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0381 - val_loss: 0.0395
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0194 - val_loss: 0.0207
Epoch 231/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0189 - val_loss: 0.0334
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0229 - val_loss: 0.0360
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0322 - val_loss: 0.0677
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0621 - val_loss: 0.0750
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0903 - val_loss: 0.0612
Epoch 236/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0861 - val_loss: 0.0264
Epoch 237/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0498 - val_loss: 0.0149
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0306 - val_loss: 0.0069
Epoch 239/1000
32/32 [==============================] - 0s 124us/

Epoch 309/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0141 - val_loss: 0.0121
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0166 - val_loss: 0.0078
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0225 - val_loss: 0.0289
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0342 - val_loss: 0.0227
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0425 - val_loss: 0.0498
Epoch 314/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0498 - val_loss: 0.0305
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0418 - val_loss: 0.0488
Epoch 316/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0385 - val_loss: 0.0280
Epoch 317/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0295 - val_loss: 0.0417
Epoch 318/1000
32/32 [==============================] - 0s 94us/s

...

Epoch 704/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0268 - val_loss: 0.0122
Epoch 705/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0258 - val_loss: 0.0160
Epoch 706/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0228 - val_loss: 0.0090
Epoch 707/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0192 - val_loss: 0.0153
Epoch 708/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0180 - val_loss: 0.0153
Epoch 709/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0233 - val_loss: 0.0561
Epoch 710/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0501 - val_loss: 0.0875
Epoch 711/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0919 - val_loss: 0.1195
Epoch 712/1000
32/32 [==============================] - 0s 94us/step - loss: 0.1075 - val_loss: 0.0535
Epoch 713/1000
32/32 [==============================] - 0s 125us/s

Epoch 783/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0474 - val_loss: 0.0702
Epoch 784/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0766 - val_loss: 0.0364
Epoch 785/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0545 - val_loss: 0.0193
Epoch 786/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0330 - val_loss: 0.0065
Epoch 787/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0169 - val_loss: 0.0050
Epoch 788/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0110 - val_loss: 0.0026
Epoch 789/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0082 - val_loss: 0.0043
Epoch 790/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0079 - val_loss: 0.0029
Epoch 791/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0087 - val_loss: 0.0077
Epoch 792/1000
32/32 [==============================] - 0s 125us

Epoch 862/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0185 - val_loss: 0.0274
Epoch 863/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0222 - val_loss: 0.0256
Epoch 864/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0267 - val_loss: 0.0467
Epoch 865/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0415 - val_loss: 0.0474
Epoch 866/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0504 - val_loss: 0.0496
Epoch 867/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0491 - val_loss: 0.0242
Epoch 868/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0296 - val_loss: 0.0172
Epoch 869/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0196 - val_loss: 0.0068
Epoch 870/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0129 - val_loss: 0.0072
Epoch 871/1000
32/32 [==============================] - 0s 93us/

Epoch 941/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0203 - val_loss: 0.0182
Epoch 942/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0246 - val_loss: 0.0157
Epoch 943/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0272 - val_loss: 0.0238
Epoch 944/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0325 - val_loss: 0.0177
Epoch 945/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0313 - val_loss: 0.0221
Epoch 946/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0320 - val_loss: 0.0139
Epoch 947/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0271 - val_loss: 0.0180
Epoch 948/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0265 - val_loss: 0.0118
Epoch 949/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0233 - val_loss: 0.0182
Epoch 950/1000
32/32 [==============================] - 0s 156us

[array([[-0.15763532, -0.2957779 , -0.3428785 ,  0.4227206 , -0.16910464],
        [ 0.14497595,  0.5587817 ,  0.48592293, -0.13342984, -0.15758625],
        [ 0.2225825 , -0.3541512 ,  0.57343173,  0.2374038 , -0.8061031 ]],
       dtype=float32),
 array([-0.5702358 ,  0.20676765, -0.29827484, -0.38649628, -0.31167793],
       dtype=float32),
 array([[-0.431567  , -0.8111289 , -0.78987247,  0.16132315, -0.19184472,
         -0.29530847, -0.19785976,  0.7799659 , -0.15891474, -0.22259642],
        [ 0.49031386,  0.35086226, -0.25271568,  0.37449893, -0.31349105,
         -0.04559926,  0.25191182, -0.15535133, -0.02649643,  0.24068083],
        [ 0.02410328, -0.15771222, -0.39661303,  0.20237395, -0.4487569 ,
         -0.50105244,  0.28005004, -0.40933287,  0.4084921 , -0.30130005],
        [ 0.42676318, -0.4013149 ,  0.24488227, -0.6704315 ,  0.41123033,
         -0.2917545 , -0.33668187,  0.54069686,  0.45476985,  0.37890977],
        [-0.7914646 , -0.00664136, -0.4404633 ,  0.3063543

In [44]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_rmsprop_3rd.h5')